In [ ]:
%matplotlib inline
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
import math
import treelib
import pyclust


In [ ]:
table1 = pd.read_csv('File11.csv')

table2 = pd.read_csv('File22.csv')


frames = [table1, table2]
result = pd.merge(table1, table2, how = 'outer', on=['doc_id', 'replydate'])
# result.head()

result['response_median_hrs'] = result['response_median']/60
result['likes'] = result['likes'].fillna(0)


result['normalised_replies'] = result['replies'].map(lambda x: (math.log(x + 2.44,9.44))*100 if ((x >= 1) and (x <= 7)) else
100 if x > 7 else 0)

result['normalised_char_count'] = result['char_count_median'].map(lambda x:  (0.00001*(x**2)+0.0002*x+0.0009)* 100 if  ((x >= 0) and (x < 125)) else
(-0.000005*(x**2)+0.0049*x-0.2885)*100 if ((x >= 125) and (x <= 500)) else 100 if x > 500 else 0)


result['normalised_response'] = result['response_median_hrs'].map(lambda x: (0.0239*(x**2)-0.2234*x+0.7869)*100 if ((x >= 0) and (x < 5)) else
(0.001*(x**2)-0.0316*x+0.3636)*100 if ((x >= 5) and (x < 15)) else (0.00009*(x**2)-0.0069*x+0.1855)*100 if ((x>= 15) and (x<40)) else 0)

result['normalised_likes'] = result['likes'].map(lambda x: (0.0043*(x**3)-0.0683*(x**2)+0.3627*x +0.3168)*100 if ((x >= 1) and (x < 8)) else 0 if x ==0 else 100)

result['DDS'] = 0.4*result['normalised_likes'] + 0.3*result['normalised_char_count'] + 0.2*result['normalised_response'] + 0.1*result['normalised_replies']

table = result[['doc_id','replydate','DDS']]

decays = pd.Series([0.017948,0.020617,0.020617,0.023683,0.027205,0.031250,0.035897,0.041235,0.047366,0.054409,0.062500,0.071794,0.082469,0.094732,0.108819,0.125000,0.143587,0.164938,0.189465,0.217638,0.250000,0.287175,0.329877,0.378929,0.435275,0.500000,0.574349,0.659754,0.757858,0.870551])

table['replydate'] = pd.to_datetime(table['replydate'])
table = table.sort(['replydate'], ascending = True)
# min_date = table['replydate'].min()
# max_date = table['replydate'].max()
# print min_date, max_date
# print table['DDS'].min()

df =pd.pivot_table(table , values = ['DDS'],index = ['replydate'], columns=['doc_id'], fill_value = '0')

stack = df.stack()
stack['DDS'] = pd.to_numeric(stack['DDS'])
stack.reset_index(inplace=True)
stack_doc = stack.groupby('doc_id',sort = True)
stack["CDS"] = 0
for x, y in stack_doc.groups.iteritems():
    doc_x = stack.ix[y]
    stack.loc[y, "CDS"] = pd.rolling_apply(doc_x['DDS'].values, 30, lambda x: sum(decays)*np.average(x, weights=decays))
stack['CDS'] = stack['CDS'].fillna(0)
print stack.tail(n = 200)

In [ ]:
stack[stack['doc_id'] == 136109]